In [1]:
import geemap
import ee

In [4]:
ee.Authenticate()
ee.Initialize(project="ee-briankipsang19")

In [15]:
import time
import pandas as pd

img = ee.Image(img)

def export_oneimage(img, folder, name, region, scale, crs):
    task = ee.batch.Export.image.toDrive(image=img, description=name, folder=folder, fileNamePrefix=name, region=region, scale=scale, crs=crs)
    task.start()
    while task.status()['state'] == 'RUNNING':
        print('Running...')
        # Perhaps task.cancel() at some point.
        time.sleep(10)
    print('Done.', task.status())

locations = pd.read_csv('/content/locations.csv')

# Define Uasin Gishu County bounding box coordinates
uasin_gishu_bbox = [[34.6898, 0.3601], [35.8078, 0.3601], [35.8078, 0.9558], [34.6898, 0.9558]]

# Transforms an Image Collection with 1 band per Image into a single Image with items as bands

def appendBand(current, previous):
    # Convert current and previous to ee.Image
    current = ee.Image(current)
    previous = ee.Image(previous)

    # Select bands if necessary
    current = current.select([0, 1, 2, 3, 4, 5, 6])  # Adjust band selection as needed

    # Add bands to the destination image
    if previous:
        accum = previous.addBands(current)
    else:
        accum = current

    return accum


imgcoll = ee.ImageCollection('MODIS/MOD09A1') \
    .filterBounds(ee.Geometry.Polygon(uasin_gishu_bbox))

img = imgcoll.first()
img = img.toShort()

for lat, lon in locations.values:
    fname = 'location_{}_{}'.format(lat, lon)

    offset = 0.11
    scale = 500
    crs = 'EPSG:4326'

    region = ee.Geometry.Rectangle([lon - offset, lat - offset, lon + offset, lat + offset])

    while True:
        try:
            export_oneimage(img, 'Data', fname, region, scale, crs)
        except Exception as e:
            print('Error:', e)
            print('Retry...')
            time.sleep(10)
            continue
        break

Done. {'state': 'READY', 'description': 'location_0.5364_35.2916', 'creation_timestamp_ms': 1707403126698, 'update_timestamp_ms': 1707403126698, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '5WZNW6IJIIA66P5WTLIQH2II', 'name': 'projects/ee-briankipsang19/operations/5WZNW6IJIIA66P5WTLIQH2II'}
Done. {'state': 'READY', 'description': 'location_0.5225_35.2478', 'creation_timestamp_ms': 1707403128057, 'update_timestamp_ms': 1707403128057, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'KTHGVN6GEDADPFFPQYNG662H', 'name': 'projects/ee-briankipsang19/operations/KTHGVN6GEDADPFFPQYNG662H'}
Done. {'state': 'READY', 'description': 'location_0.4979_35.2986', 'creation_timestamp_ms': 1707403129215, 'update_timestamp_ms': 1707403129215, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'AT2ZJZHSUP2W7IY245DOM3LI', 'name': 'projects/ee-briankipsang19/operations/AT2ZJZHSUP2W7IY245DOM3LI'}
Done. {'state': 'READY', 'description': 'location_0.6194_35.1931', 'creation_ti